# Tennis Game Results

### Introduction

The atp2017-2019.csv data set (game_stats in this notebook) shows various statistics of the top 500 players from 2017-2019. Included in this data set are the names of the tournaments, the surface of the court, number of player in the tournament, tournament level, and information on each player. The information about the players included their attributes and information on their performance during the match. In this project, we intend to predict the outcome of a player's match based on their performance during the match. We intend on using number of aces scored by the player, double faults earned, break points saved as the predictors.



### Preliminary exploratory data analysis









In [2]:
import pandas as pd

In [3]:
game_stats = pd.read_csv("data/HTRU_2.csv")
game_stats

,140.5625,55.68378214,-0.234571412,-0.699648398,3.199832776,19.11042633,7.975531794,74.24222492,0
0,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
1,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
2,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
3,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
4,93.570312,46.698114,0.531905,0.416721,1.636288,14.545074,10.621748,131.394004,0
...,...,...,...,...,...,...,...,...,...
17892,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,0
17893,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,0
17894,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,0
17895,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,0


In [50]:
game_stats.columns

Index(['Unnamed: 0', 'tourney_id', 'tourney_name', 'surface', 'draw_size',
       'tourney_level', 'tourney_date', 'match_num', 'winner_id',
       'winner_seed', 'winner_entry', 'winner_name', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'score', 'best_of', 'round', 'minutes', 'w_ace', 'w_df',
       'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved',
       'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon',
       'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 'winner_rank',
       'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [51]:
game_stats = game_stats[['winner_id', 'loser_id', 'w_ace', 'w_df',
       'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved',
       'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon',
       'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced']]
game_stats

,winner_id,loser_id,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,105453,106421,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0
1,106421,104542,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0
2,105453,104871,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0
3,104542,200282,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0
4,106421,105683,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6861,105777,105676,5.0,6.0,103.0,67.0,47.0,18.0,16.0,7.0,10.0,10.0,5.0,107.0,54.0,42.0,22.0,15.0,11.0,15.0
6862,105676,106298,12.0,0.0,74.0,48.0,40.0,17.0,14.0,0.0,0.0,6.0,2.0,95.0,53.0,33.0,21.0,14.0,6.0,11.0
6863,104542,104327,12.0,2.0,64.0,33.0,31.0,19.0,12.0,0.0,0.0,1.0,2.0,81.0,54.0,30.0,12.0,12.0,5.0,11.0
6864,105676,104542,1.0,0.0,122.0,71.0,47.0,33.0,15.0,10.0,10.0,10.0,3.0,88.0,54.0,44.0,13.0,14.0,7.0,10.0


### Methods





### Expected outcomes and significance







